In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import sys
sys.path.append('../..')
from laplace_coeff_Thomas import *
from datanew import *

# needed for the laplace coefficients from the lookup table
cfs=LaplaceCoeffs()
dcfs=LaplaceCoeffsDerivatives()

In [2]:
# Konstanten Einheitenlos
M_STAR=1
MU1=3e-3
MU2=1e-3
M1=MU1*M_STAR
M2=MU2*M_STAR
mmr=2
G=1
R_0=1 #1 au

SIGMA_0=1e-3
SIGMASLOPE=1.5

ALPHAVISCOSITY=1e-5       
TSLOPE=0.5           
FLARINGINDEX=0.25        
H_0=0.06
SIGMARATIO=0.3   #for kanagawa

R_IN=np.arange(1.1,3 - 0.002,0.001)
R_OUT=R_IN *mmr**(2/3)

In [3]:
#width of the gap

def deltain(r,mu,Sigmaratio):
    b= (0.5*Sigmaratio+0.16)
    
    c=(mu**2)
    c*=(aspect_ratio(r)) **(-3)
    c*=ALPHAVISCOSITY**(-1)

    return r*b*c**(0.25) 

In [4]:
# Functions for the Torque of ward3
def omega_kepler(r):
    return np.sqrt(G*M_STAR/(r**3))

def surface_density(r):
    return SIGMA_0* (r/R_0)**(-SIGMASLOPE)

def aspect_ratio(r):
    return H_0 *(r/R_0)**FLARINGINDEX

#cutoff radius
def r_lim_k(r):
    return r-aspect_ratio(r)*r *2/3 ,r+aspect_ratio(r)*r *2/3



def vk(r):
    return np.sqrt(G*M_STAR/r)

def cs(r):
    return aspect_ratio(r)*vk(r)

def press(r):
    return surface_density(r)*cs(r)**2

def dfdlnr(f,r,*args):
    rp       = r * (1+1e-5)
    lnr      = np.log(r)
    lnrp     = np.log(rp)
    return (f(rp,*args)-f(r,*args))/(lnrp-lnr)
                                     
def dlnsigmadlnr(r):
    return dfdlnr(sigma,r)/sigma(r)
                                     
def dlnpdlnr(r):
    return dfdlnr(press,r)/press(r)
                                     
def omphi(r):
    return omega_kepler(r) * np.sqrt(1+(cs(r)**2/vk(r)**2)*dlnpdlnr(r))
                                     
def kappa(r):
    qkap = -dfdlnr(omphi,r)/omphi(r)
    return np.sqrt(2*(2-qkap))*omphi(r)
                                     
def Dstar(r,r0,m):
    return kappa(r)**2-m**2*(omphi(r)-omega_kepler(r0))**2+(m*cs(r)/r)**2
                                     
                                     
def alphar(r,r0):
    return r/r0

cfs=LaplaceCoeffs()
                                     
def phim(r,r0,m,mp):
    
    laplace = np.zeros((len(r0),len(m)))
        
    for i in np.arange(len(m)):
        for j in np.arange(len(r0)):
            laplace[j][i] = cfs( r[j][i]/r0[j], m[i])
    
    return -(G*mp/r0)*laplace
                                     
def xi(r,m):
    return m*cs(r)/(r*kappa(r))
                                     
def ff(r,r0,m):
    om = omphi(r)
 
    return  m*(om-omega_kepler(r0))/om
                                     
def Psim(r,r0,m,mp):
    return (dfdlnr(phim,r,r0[:,None],m,mp)+2*m*ff(r,r0[:,None],m)*phim(r,r0[:,None],m,mp))* 1/np.sqrt(1+4*xi(r,m)**2)
                                     


In [5]:
#load the locations of D_K

Kley_in=np.loadtxt(f"/home/kim/Dokumente/Uni/Bachelor/Migration/Neuer_Start/D_K_locations_in{mmr}_{H_0}_{FLARINGINDEX}.dat").reshape(len(np.arange(1,20)),len(np.arange(1.1,3 - 0.002,0.001)))
Kley_out=np.loadtxt(f"/home/kim/Dokumente/Uni/Bachelor/Migration/Neuer_Start/D_K_locations_out{mmr}_{H_0}_{FLARINGINDEX}.dat").reshape(len(np.arange(1,20)),len(np.arange(1.1,3 - 0.002,0.001)))


Kley_in=Kley_in.T
Kley_out=Kley_out.T

In [6]:
#cufoff radius
def r_lim_k(r):
    return r-aspect_ratio(r)*r *2/3 ,r+aspect_ratio(r)*r *2/3

# Matrix of Lindblad locations with the consideration of the gap width and the cutoff radius
def r_Llist(m,rp):
    
    rvin = Kley_in
    rvin=np.where(rvin==0, np.nan, rvin)
    rvout = Kley_out
    
    r1, r2 = r_lim_k(rp)
    r1, r2 = r1[:,None], r2[:,None]
    

    delta_in = deltain(rp, MU1, SIGMARATIO)/2
    delta_out = deltain(rp, MU2, SIGMARATIO)/2
   
    finalin = np.where(rvin  < (rp - delta_in)[:,None], rvin, np.nan)
    finalin =  np.where(finalin != np.nan,np.where(finalin >= r1, r1, finalin),np.nan)
    
    finalout= np.where(rvout > (rp + delta_out)[:,None], rvout, np.nan)
    finalout= np.where(finalout != np.nan,np.where(finalout <= r2, r2, finalout), np.nan)
        
    return finalin, finalout

In [9]:
# Value of Lindblad locations with the consideration of the gap width and the cutoff radius
def r_Lvalue(m,rp_index,rp):
    
    rvin = Kley_in[rp_index][m-1]
    rvin=np.where(rvin==0, np.nan, rvin)
    rvout = Kley_out[rp_index][m-1]
    
    r1, r2 = r_lim_k(rp)
    
    delta_in = deltain(rp, MU1, SIGMARATIO)/2
    delta_out = deltain(rp, MU2, SIGMARATIO)/2
   
    finalin = np.where(rvin  < (rp - delta_in), rvin, np.nan)
    finalin =  np.where(finalin != np.nan,np.where(finalin >= r1, r1, finalin),np.nan)
    
    finalout= np.where(rvout > (rp + delta_out), rvout, np.nan)
    finalout= np.where(finalout != np.nan,np.where(finalout <= r2, r2, finalout), np.nan)
        
    return finalin, finalout



In [7]:
#Torque of ward3
def T(m,mp,r0): #mp ist nicht dimensionslos
    rvin=r_Llist(m,r0)[0]
    rvout=r_Llist(m,r0)[1]

    T_in=np.pi**2*m*surface_density(rvin)*Psim(rvin,r0,m,mp)**2/dfdlnr(Dstar,rvin,r0[:,None],m)
    T_out=np.pi**2*m*surface_density(rvout)*Psim(rvout,r0,m,mp)**2/dfdlnr(Dstar,rvout,r0[:,None],m)
    
    T_in=np.nan_to_num(T_in)
    T_out=np.nan_to_num(T_out)
    
    return T_in, T_out

In [10]:
#Thomas section wise kanagawa profile
def Kanagawa17Profile(r, rp, Mp, ALPHAVISCOSITY):
    """ Gap profile from Kanagawa et al. 2017 Eq. (6).
    
    This function calculates the gap shape Sigma(r)/Sigma0.
8
    Parameters
    ----------
    r : array
        Grid radii.
    rp : float
        Orbital radius of the planet.
    Mp : float
        Planet mass in units of stellar mass.
    h : float
        Aspect ratio (H/R) at the location of the planet.
    alpha : float
        Viscous alpha parameter.
        
    Returns
    -------
    array
        Reduction factor of the surface density.
    
    """
    Sigma = np.ones_like(r)
    
    h=aspect_ratio(r)
    Kp = Mp**2 * h**(-3) * ALPHAVISCOSITY**(-1) # Eq. (2)
    Kp14 = Kp**(1/4)
    K = Mp**2 * h**(-5) *  ALPHAVISCOSITY**(-1) # Eq. (11)
    
    SigmaMin = 1 / (1+0.04*K) # Eq. (10)
    
    DR1 = (SigmaMin/4 + 0.08)*Kp14*rp # Eq. (8)
    DR2 = 0.33*Kp14*rp # Eq. (9)
    
    # regions according to Eq. (6)
    Dr = np.abs(r - rp)
    region1 = Dr <= DR1
    region2 = np.logical_and(Dr > DR1, Dr < DR2)
    
    # surface denity ratio according to Eq. (6)
    Sigma[region1] = SigmaMin*0.4
    Sigma[region2] = 4/Kp14 * Dr[region2]/rp - 0.32
    
    # The third region for Dr > DR2 is just Sigma0, thus it stays 1 here.
    
    return Sigma

In [11]:
#Inner Torques with the weigtening of the Kanagwa profile

def Torque_in_all(m_max):
    torque_in=[]
    
    m=np.arange(1,m_max,1)
    
    Kanagawa_in=[[Kanagawa17Profile(r_Lvalue(i,j,R_IN[j])[1], R_IN[j], M1, ALPHAVISCOSITY) for i in m]for j in np.arange(0,len(R_OUT))] 
    T_in=T(m,M1,R_IN)[0]
    torque_in = np.sum(T_in*Kanagawa_in, axis=1)
    
    return torque_in

In [12]:
#Outer Torques with the weigtening of the Kanagwa profile
def Torque_out_all(m_max):
    torque_out=[]
    m_k=np.arange(1,m_max,1)
    
    Kanagawa_out=[[Kanagawa17Profile(r_Lvalue(i,j,R_OUT[j])[1], R_OUT[j], M2, ALPHAVISCOSITY) for i in m_k]for j in np.arange(0,len(R_OUT))] 
    T_out=T(m_k,M2,R_OUT)[1]
    torque_out = np.sum(T_out*Kanagawa_out,axis=1)

    return torque_out


In [14]:
#Plot der Torques

plt.figure(figsize=(10,5))
plt.plot(R_IN,Torque_in_all(20),label='all inner Torques')
plt.plot(R_OUT,Torque_out_all(20),label='all outer Torques')


plt.xlabel('r',fontsize=13)
plt.ylabel('Torques',fontsize=13)
plt.title('all Torques',fontsize=13)
plt.axhline(0, c = "k", linestyle = "--")
plt.legend(fontsize=13)

# Migrationsraten

In [15]:
#Migration rate which results of the angular momentum calculation

#hier mp'= M1+M2
def rate(T,a,M1,M2):
    Sigma= np.sqrt(G*M_STAR/a**3)
    
    rate=2*T/(Sigma*a*(M1+M2))
    return rate

In [16]:
#Sum over the rates to plot them

T_allgew=[sum(x) for x in zip(Torque_in_all(20),Torque_out_all(20))]

rate_ingew= [rate(T,r,M1,M2) for T,r in zip(Torque_in_all(20),R_IN)]
rate_outgew= [rate(T,r,M1,M2) for T,r in zip(Torque_out_all(20),R_OUT)]
rate_allgew=[rate(T,r,M1,M2) for T,r in zip(T_allgew,R_IN)]

KeyboardInterrupt: 

In [ ]:
#Plot the rates
plt.figure(figsize=(15,7))
plt.plot(R_IN,rate_ingew  , label='Migration rate inner Torques')
plt.plot(R_OUT,rate_outgew, label='Migration rate outer Torques')
plt.plot(R_IN,rate_allgew, label='Migration rate insgesamt')
plt.xlabel('r',fontsize=13)
plt.ylabel('Migration rate',fontsize=13)
plt.legend(fontsize=13)
plt.axhline(0, c = "k", linestyle = "--")
plt.grid()
plt.minorticks_on()
#plt.savefig(f'Ward3_D_K_pictures/rates_ward3_D_K_{FLARINGINDEX}_{SIGMA_0}_{ALPHAVISCOSITY}_{M1}_{SIGMARATIO}_{H_0}.png')

In [ ]:
#Abspeichern der Daten
orte=BM08newortederrate
index=[int(np.round((i-1.1)/0.001)) for i in orte]
    

file2write=open(f"BM08Ward3_D_K_{mmr}_{SIGMARATIO}.txt",'a+')
file2write.write('newtheory_h0_006_alpha_1e_5_planets_3_1_sigma0_1e_3'+' ' )

file2write.write(str(rate_allgew[index[9]])+' ')


file2write.write(str(SIGMARATIO)+' ')
file2write.write(str(M1)+' ')
file2write.write(str(FLARINGINDEX)+' ')
file2write.write(str(R_IN[index[9]])+' ')
    
file2write.write('\n')
file2write.close()